## 梯度的问题

有时候在计算中，我们如果自定义了一些函数，很有可能会出现梯度崩掉的情况，通常是梯度爆炸（梯度NaN）或者梯度消失（0）

下面先看梯度爆炸的例子，主要参考了：https://zhuanlan.zhihu.com/p/79046709

其他资料：

- https://blog.csdn.net/mch2869253130/article/details/111034068

它是一个loss函数中有幂函数的情况，即 $\Gamma(x_i) = x ^ r$，其中r是0到1之间的数

这个loss函数在反向传播过程中很可能会遇到梯度爆炸，因为反向传播的过程是对loss链式求一阶导数的过程：

$$\frac{d\Gamma(x_i)}{dx_i}=\frac{r}{{x_i}^{1-r}}$$

出现了 1/x 的情况，这就会出现梯度崩掉的情况，为了避免这种情况，手动设置下条件，让$\Gamma(x_i)$变成个条件判断的函数，它是这样定义的，就是 x<0.003时候，定义成12.9 * x，其余时候还是原函数，按理说，就不会有崩掉的情况了。

In [1]:
import torch

In [2]:
"""
loss = mse(X, gamma_inv(X))
"""
def loss_function(x):
    mask = (x < 0.003).float()
    print("mask:", mask)
    gamma_x = mask * 12.9 * x + (1-mask) * (x ** 0.5)
    loss = torch.mean((x - gamma_x) ** 2)
    return loss

In [3]:
x = torch.tensor([0, 0.0025, 0.5, 0.8, 1], requires_grad = True)
loss = loss_function(x)
print('loss:', loss)
loss.backward()
print(x.grad)

mask: tensor([1., 1., 0., 0., 0.])
loss: tensor(0.0105, grad_fn=<MeanBackward0>)
tensor([    nan,  0.1416, -0.0243, -0.0167,  0.0000])


改进后的公式是一个分支结构，在实现时，就采用了类似于Matlab中矩阵计算的mask方式，满足条件的$x_i$在mask中对应位置的值为1，因此， 这个公式的结构只会保留 x<0.003 的结果，同样的道理， 1-mask 就保留另一部分，合一块就实现了上述改进后的公式。但是从实现的情况看，显然不是这样的，nan还是有，结果还是崩了。

换成where语句了也是一样的，还是会有这个问题，可能是where的具体实现就是上面这样mask来实现的。

In [4]:
def loss_function1(x):
    gamma_x = torch.where(x < 0.003, 12.9 * x, x ** 0.5)
    loss = torch.mean((x - gamma_x) ** 2)
    return loss

In [5]:
x1 = torch.tensor([0, 0.0025, 0.5, 0.8, 1], requires_grad = True)
loss1 = loss_function1(x1)
print('loss:', loss1)
loss1.backward()
print(x1.grad)

loss: tensor(0.0105, grad_fn=<MeanBackward0>)
tensor([    nan,  0.1416, -0.0243, -0.0167,  0.0000])


上面的过程在Python解释器中解释或许是这样的：

1. 计算 mask * 12.9的时候是对mask进行广播式的乘法，结果为：原本为1的位置变为了12.9，原本为0的位置依旧为0；
2. 将1.的结果继续与x相乘，本质上仍然是与x的每个元素相乘，只是mask中不满足条件的 $x_i$ 位置为0，表现出的结果是仅对满足条件的 $x_i$  进行了计算；
3. 同理，$\Gamma(x_i)$公式的后半部分也是同样的计算过程，即，x  中的每个值依旧会进行 $x^{0.5}$ 的计算；

按照上述过程进行前向传播，在反向传播时，梯度不是从某一个分支得到的，而是两个分支相加得到的，换句话说，依旧没能解决梯度变为nan的问题。

所以问题是 $x_i$=0 依旧参与了幂次运算，导致在反向传播时计算出的梯度为nan。

要解决这个问题，就要保证在 $x_i=0$ 时不会进行这样的计算。

新的代码如下：

In [6]:
def loss_function2(x):
    mask = x < 0.003
    print("mask:", mask)
    gamma_x = torch.zeros(x.size())
    gamma_x[mask] = 12.9 * x[mask]
    mask = x >= 0.003
    gamma_x[mask] = x[mask] ** 0.5
    loss = torch.mean((x - gamma_x) ** 2)
    return loss

In [7]:
x2 = torch.tensor([0, 0.0025, 0.5, 0.8, 1], requires_grad = True)
loss2 = loss_function2(x2)
print('loss:', loss2)
loss2.backward()
print(x2.grad)

mask: tensor([ True,  True, False, False, False])
loss: tensor(0.0105, grad_fn=<MeanBackward0>)
tensor([ 0.0000,  0.1416, -0.0243, -0.0167,  0.0000])


可以看到，这时候梯度不再为nan了。因为这里改变了对于 $\Gamma(x_i)$ 分支的处理方式，先是构建了mask，但是随后不是一次性让gamma_x得到计算，而是分开，小于的单独计算一波，大于的再计算一波。这样等于0的那部分就没参与到幂次运算中，这样求导的时候不会有问题。